<a href="https://colab.research.google.com/github/lyuoveta/git_test_model_1000/blob/master/Llama/Llama_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.0.2-py3-none-any.whl (819 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.27.7-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.0 MB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
    Using cached distro-1.8.0-py3-none-any.whl (20 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.42.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-dr33gnsh/overlay/local/bin
    changing mode of /tmp/pip-build-env-dr33gnsh/overlay/local/bin/ninja to 755
    changing mod

In [4]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [6]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [7]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [8]:

from google.colab import files


pd= files.upload()

Saving tagged_data_llama_256.csv to tagged_data_llama_256 (1).csv


In [9]:
import pandas as pd
import random
df = pd.read_csv('tagged_data_llama_256.csv')


In [10]:
df = df[['reviews', 'Correct review ', 'Incorrect review ']]
df.head()

,reviews,Correct review,Incorrect review
0,Not bad but could be better,1,0
1,This is a test review post done by Amar 4,1,0
2,This is a test review post done by Amar 3,1,0
3,Taking biotin 5000mcg from October first week ...,1,0
4,My nails grew in noticeably thicker and strong...,1,0


In [11]:
df_incorrect = df.loc[df['Incorrect review '] == 1]['reviews'].reset_index(drop=True)
df_correct = df.loc[df['Correct review '] == 1]['reviews'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'While I didnt take melatonin SOLELY for nicotine withdrawl, I took Chantix....I found melatonin effective for sleep.....seemed that Chantix kept me from sleeping, esp that first 2 weeks..'

In [12]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'I take 2000mg of V -D and my numbers are going down  how many mg is safe to take, or should I see why my V -D is it going down'

In [13]:
#Step 6: Generating the Response

In [25]:
for _ in range(20):
    prompt = f'''Please evaluate the following review for correctness. A '1' indicates that the "
            "review is correct (it is not spam and contains information about the product effects), "
            "while a '0' indicates an incorrect review (it is spam or doesn't contain information about the product effects):
    1. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    2. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    3. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    4. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    5. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1

    Please provide the correctness of the following review by entering 0 for incorrect or 1 for correct:
    "{df['reviews'][random.choice(range(len(df)))]}"
    '''
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.

    USER: {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.2, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)
    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    matches = re.findall(r'ASSISTANT:.*?([01])', response_text, re.DOTALL)

    print(matches)



Llama.generate: prefix-match hit


{'id': 'cmpl-2c6a4c72-8cfc-4dea-aef1-b2ca641d6b38', 'object': 'text_completion', 'created': 1701276470, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I use Berberine to lower my sugar levels.  It works great and has no side effects like Metformin and works much better.\n     ANSWER: 1\n    2. GREAT.\n     ANSWER: 0\n    3. It has helped me.\n     ANSWER: 1\n    4. Vitamin D in Nature\'s Way or Now Foods - they have no coloring. Best

Llama.generate: prefix-match hit


{'id': 'cmpl-2b366dfe-1aab-4c58-903f-c60db9c436e6', 'object': 'text_completion', 'created': 1701276505, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I suffer from cold sores and nothing has helped. I tried CS and it was remarkable. A few sprays on the sore 3-4 times a day and the last sore was gone in a day and a half.\n     ANSWER: 1\n    2. This is easy to use!\n     ANSWER: 0\n    3. Been taking the suggested amount for almost 1

Llama.generate: prefix-match hit


...

{'id': 'cmpl-57147cc2-e2e3-41b2-89ce-bd39c1453d55', 'object': 'text_completion', 'created': 1701277420, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I feel like Melatonin helps me fall to sleep easier, but I still have trouble staying asleep.\n     ANSWER: 1\n    2. i have no pain , no fracture ,my vit D level is impruved , my bondensity also .\n     ANSWER: 0\n    3. good\n     ANSWER: 1\n    4. skin cancer no contest silver kille

Llama.generate: prefix-match hit


{'id': 'cmpl-e3f308ed-f07d-4e2f-8ca5-6ca7bc065e98', 'object': 'text_completion', 'created': 1701277439, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. my elbows arent scally anymore,i have very low amounts of d in my blood i take 10,000iu a week and im still low\n     ANSWER: 1\n    2. Ginko iks supposed to help with dizziness,but side effects can cause diziness,confusing.\n     ANSWER: 0\n    3. this treatment seem somewhat effectiv

Llama.generate: prefix-match hit


{'id': 'cmpl-883f4c9d-810e-48fd-a8f8-7a4e096292a5', 'object': 'text_completion', 'created': 1701277537, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I used this when I wasn\'t having the best results with my prescription medicine.  It worked very well.\n     ANSWER: 1\n    2. NOT to be taken with Zeloda (capecitabine).\n     ANSWER: 0\n    3. dr. gave me 50K to start . this caused severe reflux and ringing in ears. have to go off t

In [26]:
!pip install scikit-learn

true_labels = [1, 0, 0, 0, 1,1,0]
predicted_labels = [0,0,0,1,1,1,0]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")


Точность модели: 71.43%
